In [1]:
# Standard library imports
import requests

# Third-party imports
import polars as pl

# Local imports
from op_analytics.datapipeline.chains.load import load_chain_metadata
from op_analytics.datasources.coingecko.price_data import CoinGeckoDataSource
from op_analytics.datasources.coingecko.dataaccess import CoinGecko
from op_analytics.coreutils.request import new_session

In [2]:
# 1. Load chain metadata to get token IDs
from op_analytics.datapipeline.chains.load import load_chain_metadata

# Load chain metadata
chain_metadata = load_chain_metadata()

# Get unique non-null CoinGecko API keys
token_ids = (
    chain_metadata.filter(pl.col("cgt_coingecko_api").is_not_null())
    .select("cgt_coingecko_api")
    .unique()
    .to_series()
    .to_list()
)

print(f"Found {len(token_ids)} unique tokens with CoinGecko API keys")

2025-06-17 13:33:08 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=94000
2025-06-17 13:33:08 [debug    ] loaded vault: 28 items         filename=vault.py lineno=79 process=94000
2025-06-17 13:33:08 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=94000
2025-06-17 13:33:10 [info     ] Loaded OP chains metadata from /Users/michaelsilberling/Documents/GitHub/op-analytics/op_chains_tracking/inputs/chain_metadata_raw.csv filename=load.py lineno=87 process=94000
2025-06-17 13:33:10 [info     ] [REPO vs. GSHEETS] ERROR: Chain Metadata is different filename=load.py lineno=99 process=94000
Found 12 unique tokens with CoinGecko API keys


In [3]:
# 2. Initialize the data source
from op_analytics.datasources.coingecko.price_data import CoinGeckoDataSource
from op_analytics.coreutils.request import new_session

# Create a session with debug logging
session = requests.Session()
data_source = CoinGeckoDataSource(session=session)

2025-06-17 13:33:10 [info     ] Using CoinGecko Free API (rate limited) filename=price_data.py lineno=90 process=94000


In [4]:
# 3. Fetch price data for all tokens
# Fetch 7 days of data for all tokens
all_prices = []
for token_id in token_ids:
    try:
        prices = data_source.get_token_prices(token_ids=[token_id], days=365)
        all_prices.append(prices)
        print(f"Successfully fetched data for {token_id}")
    except Exception as e:
        print(f"Error fetching data for {token_id}: {e}")

# Combine all price data
prices_df = pl.concat(all_prices)
print(f"\nSuccessfully fetched price data for {len(prices_df)} token-days")
print("\nSample of fetched data:")
print(prices_df.head())

2025-06-17 13:33:10 [info     ] Received data for tokens       count=1 filename=price_data.py lineno=163 process=94000
Successfully fetched data for settled-ethxy-token
2025-06-17 13:33:22 [info     ] Received data for tokens       count=1 filename=price_data.py lineno=163 process=94000
Successfully fetched data for automata
2025-06-17 13:33:34 [info     ] Received data for tokens       count=1 filename=price_data.py lineno=163 process=94000
Successfully fetched data for frax-ether
2025-06-17 13:33:46 [warning  ] stamina.retry_scheduled        args=('<op_analytics.datasources.coingecko.price_data.CoinGeckoDataSource object at 0x13e24cda0>',) callable=op_analytics.datasources.coingecko.price_data.CoinGeckoDataSource.get_token_prices caused_by="CoinGeckoRateLimit('Rate limit exceeded: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/mantle/market_chart?vs_currency=usd&days=90&interval=daily')" filename=_structlog.py kwargs={'token_ids': ['mantle'], 'day

In [5]:
# 4. Write to BigQuery
from op_analytics.datasources.coingecko.dataaccess import CoinGecko

# Write the data to BigQuery
CoinGecko.DAILY_PRICES.write(prices_df)
print("Successfully wrote data to BigQuery")

2025-06-17 13:36:58 [info     ] done writing 10rows 2.6KB      filename=gcs_parquet.py lineno=57 maxrss=203489280 path='/Users/michaelsilberling/Documents/GitHub/op-analytics/ozone/warehouse/coingecko/fact_coingecko_daily_prices_v1/dt=2025-03-19/out.parquet' process=94000 root=coingecko/fact_coingecko_daily_prices_v1 rows=10 size=2587
2025-06-17 13:36:58 [info     ] memory usage                   filename=gcs_parquet.py lineno=65 max_rss=203.49 process=94000 root=coingecko/fact_coingecko_daily_prices_v1
2025-06-17 13:36:59 [debug    ] done writing coingecko/fact_coingecko_daily_prices_v1 to LOCAL filename=writemanager.py lineno=131 process=94000 root=coingecko/fact_coingecko_daily_prices_v1
2025-06-17 13:36:59 [info     ] done writing 10rows 2.6KB      filename=gcs_parquet.py lineno=57 maxrss=203489280 path='/Users/michaelsilberling/Documents/GitHub/op-analytics/ozone/warehouse/coingecko/fact_coingecko_daily_prices_v1/dt=2025-03-20/out.parquet' process=94000 root=coingecko/fact_coingec

In [6]:
# 5. Read back from production to verify
df = CoinGecko.DAILY_PRICES.read_polars(min_date=prices_df["dt"].min())
print("\nData from production:")
print(df.head())

2025-06-17 13:36:59 [info     ] initialized duckdb at /var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/4by8zxu8/op-analytics.duck.db filename=client.py lineno=118 process=94000
2025-06-17 13:36:59 [info     ] create duckddb gcs secret      filename=client.py lineno=66 process=94000
2025-06-17 13:37:00 [info     ] Reading data from 'coingecko/fact_coingecko_daily_prices_v1' with filters min_date=2025-03-19, max_date=None, date_range_spec=None filename=dailydata.py lineno=94 process=94000
2025-06-17 13:37:00 [info     ] querying markers for 'coingecko/fact_coingecko_daily_prices_v1' DateFilter(min_date=datetime.date(2025, 3, 19), max_date=None, datevals=None) filename=dailydataread.py lineno=53 process=94000
2025-06-17 13:37:00 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=94000
2025-06-17 13:37:00 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=94000
2025-06-17 13:37:00 [info     ] 8 markers found       

In [8]:
# 6. Optional: Analyze the data
# Get the date range of the data
print(f"Overall date range: {prices_df['dt'].min()} to {prices_df['dt'].max()}")

# Count unique tokens
print(f"Number of unique tokens: {prices_df['token_id'].n_unique()}")

# Get date range and average price per token
token_stats = prices_df.group_by('token_id').agg([
    pl.col('price_usd').mean().alias('avg_price'),
    pl.col('dt').min().alias('min_date'),
    pl.col('dt').max().alias('max_date'),
    pl.col('dt').count().alias('days_of_data')
]).sort('token_id')

print("\nStats per token:")
print(token_stats)

Overall date range: 2025-03-19 to 2025-06-17
Number of unique tokens: 10

Stats per token:
shape: (10, 5)
┌─────────────────────┬─────────────┬────────────┬────────────┬──────────────┐
│ token_id            ┆ avg_price   ┆ min_date   ┆ max_date   ┆ days_of_data │
│ ---                 ┆ ---         ┆ ---        ┆ ---        ┆ ---          │
│ str                 ┆ f64         ┆ str        ┆ str        ┆ u32          │
╞═════════════════════╪═════════════╪════════════╪════════════╪══════════════╡
│ automata            ┆ 0.054153    ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ binancecoin         ┆ 626.539114  ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ binary-holdings     ┆ 0.021033    ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ celo                ┆ 0.342097    ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ frax-ether          ┆ 2123.987747 ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ gameswift           ┆ 0.013044    ┆ 2025-03-19 ┆ 2025-06-17 ┆ 91           │
│ hashkey-ecopoints   ┆ 0